# Recuperação da Informação e Busca na Web - 2018.1

### Atividade: Lab 02 - Parte 2 - PageRank
### Aluno: Johanny de Lucena Santos

## - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 

Nesta atividade você vai exercitar o algoritmo PageRank. De forma específica, você vai fazer o seguinte:

<ul>
    <ul>
        <li>Assistir o vídeo e (re)implementar o PageRank como descrito neste vídeo (https://www.youtube.com/watch?v=zv4OVNWfVt4).

<ul>
    <li>É importante que você realmente digite o código durante o vídeo (pausando quando necessário), isso é mais efetivo para o aprendizado do que somente copiar e colar do github por exemplo. Tente entender o código, ele é bem próximo ao que foi explicado em sala de aula.</li>

<li>Essa é somente uma das muitas implementações do PageRank disponíveis na Web. Você pode escolher outra se assim desejar.</li></ul>
<p>   
<li>Aplique sua implementação nos dados disponíveis em http://snap.stanford.edu/data/soc-sign-bitcoinotc.html. Esses dados representam links entre investidores de bitcoin. Cada investidor pode receber uma nota de acordo com outras pessoas que fizeram transações com ele(a). **Vamos considerar somente as pessoas que receberam notas iguais ou maior a oito. Ou seja, somente as arestas que tenham peso maior ou igual à oito**.
    <p>
<li>Uma vez calculado o PageRank, gere visualizações dos resultados usando a ferramenta Gephi (https://gephi.org/).
   <br> 
    <br>
Feitos os passos acima, responda as seguintes perguntas:
<p>
<li>Quantas iterações o PageRank precisou rodar até atingir convergência?</li>
<li>Quais os 5 investidores mais importantes segundo o PageRank? Quais seus valores de PageRank?</li>
<li>Como você poderia usar o PageRank caso você fosse um investidor em bitcoins?</li>

### Source (citation)

S. Kumar, F. Spezzano, V.S. Subrahmanian, C. Faloutsos. Edge Weight Prediction in Weighted Signed Networks. IEEE International Conference on Data Mining (ICDM), 2016.

### Importações

In [1]:
import numpy as np
import pandas as pd

## Definição do dataset a ser utilizado
Aqui eu defino o conjunto de dados a ser utilizado, incluindo os nomes das colunas, bem como removendo a coluna TIME, que será desnecessária para as manipulações do conjunto de dados.

In [2]:
dataset = pd.read_csv('../data/soc-sign-bitcoinotc.csv')
dataset.columns = ['SOURCE_ID', 'TARGET_ID', 'RATING', 'TIME']
dataset = dataset.drop('TIME', axis=1)

### Prévia do dataset

In [3]:
dataset.head()

,SOURCE_ID,TARGET_ID,RATING
0,6,5,2
1,1,15,1
2,4,3,7
3,13,16,8
4,13,10,8


### Filtragem dos dados
Vamos considerar somente as pessoas que receberam notas iguais ou maior a oito. Ou seja, somente as arestas que tenham peso maior ou igual à oito (filtrar os dados para os elementos que possuem a coluna Rating maior ou igual a 8)

In [4]:
dataset    = dataset.loc[dataset['RATING'] >= 8]
source_ids = list(dataset['SOURCE_ID'])
target_ids = list(dataset['TARGET_ID'])
nodes_ids  = list(set(source_ids).union(set(target_ids)))

dataset.head()

,SOURCE_ID,TARGET_ID,RATING
3,13,16,8
4,13,10,8
9,21,1,8
10,21,10,8
11,21,8,9


## Criando grafo de interação entre investidores de Bitcoins

In [5]:
grafo = {node:[] for node in nodes_ids}

for i in range(len(source_ids)):
    src = source_ids[i]
    tgt = target_ids[i]
    grafo[src].append(tgt)

## Gerando a matriz de adjacência
Criando a matriz H inicialmente zerada, e em seguida, preenchemos com 1 todos que possuem links de saída, dividindo-o pelo número de links que saem dele, ou seja, o número de pessoas que investiram dele.

In [7]:
h = [[0 for i in range(len(nodes_ids))] for j in range(len(nodes_ids))]

for i in range(len(nodes_ids)):
    for j in range(len(nodes_ids)):
        src = nodes_ids[i]
        tgt = nodes_ids[j]
        links_saida = len(grafo[src])
        
        if (tgt in grafo[src]):
            h[j][i] = float(1)/float(links_saida)
        else:
            h[j][i] = 0
            
h = np.matrix(h)

### Calculo do PageRank
Aqui, definimos o fator d = 0.15 e criamos a matriz B com todos os valores 1, e depois multiplicando por 1/n, onde n é o número de nós. Em seguida, calculamos o PageRank utilizando M = (1-d)*H + d*B

In [8]:
d = 0.15
fator = float(1)/float(len(nodes_ids))

#Matriz temporaria com todos os valores em 1
matriz_temp = np.matrix(
    [[1 for i in range(len(nodes_ids))] for j in range(len(nodes_ids))]
)

b = fator * matriz_temp

#Calculo do PageRank
m = (1-d)*h + d*b

#Matriz temporaria com uma coluna com valores 1
matriz_temp2 = np.matrix([[1] for i in range(len(nodes_ids))])

#Vetor v normalizado
v = fator * matriz_temp2

### Algoritmo PageRank
Algoritmo utilizado para calcular todas as interações do PageRank. Ele calcua m * v até convergir. A diferença de convergência (soma da diferença de todos os elementos) é definida como 0.001. 

In [9]:
count = 0

def pageRank(v):
    
    global count
    
    count += 1
    
    if sum(abs(m*v-v)) > 0.001:
        return pageRank(m*v)
    else:
        return m*v

### Gerando o resultado do PageRank
Aqui usamos o algoritmo do PageRank para verificar o pageRank final do conjunto de dados. 
<p>
Inicialmente, criamos o dataframe referente aos resultados do PageRank.

In [21]:
resultado = pageRank(v)
resultado = [cell.item(0,0) for cell in resultado]

pagerank_result = pd.DataFrame({'NODE_ID':nodes_ids, 'PAGE_RANK':resultado})

### Criação do arquivo csv para visualização dos resultados
Esses arquivos csv gerados a partir do dataframe dos resultados do PageRank e do dataset filtrado serão utilizados para a visualização dos resultados na ferramenta **Gephi**.

In [30]:
pagerank_result.to_csv(path_or_buf='../Lab02/resultado/pagerank_result.csv', index=False)
dataset.to_csv(path_or_buf='../Lab02/resultado/soc-sign-bitcoinotc-filtrado.csv', columns=['SOURCE_ID', 'TARGET_ID'], index=False)

In [31]:
pagerank_df = pd.read_csv('../Lab02/resultado/pagerank_result.csv')
bitcoinotc_df =  pd.read_csv('../Lab02/resultado/soc-sign-bitcoinotc-filtrado.csv')

In [32]:
pagerank_df.head()

,NODE_ID,PAGE_RANK
0,1,0.000139
1,2,0.000028
2,4098,0.000003
3,4,0.000037
4,3,0.000004


In [34]:
bitcoinotc_df.head()

,SOURCE_ID,TARGET_ID
0,13,16
1,13,10
2,21,1
3,21,10
4,21,8


## Questões

### 1) Quantas iterações o PageRank precisou rodar até atingir convergência?
Para saber o número de iterações necessárias que o PageRank precisou rodar até atingir a convergência, basta imprimir o valor da variável global **count** usada na função **pageRank(v)**, como vemos à seguir.

In [14]:
print("Número de iterações", count)

Número de iterações 35


### 2) Quais os 5 investidores mais importantes segundo o PageRank? Quais seus valores de PageRank?
Para responder à essa questão, basta ordenar os valores gerados de forma decrescente do valor de PageRank. Como vemos na tabela abaixo, na coluna NODE_ID temos o ID do investidor, e em seguida, vemos o seu valor de PageRank.
<p>
Para o investidor com maior PageRank, no caso o investidor com **ID=1**, seu PageRank vale **0.000139**. O restante pode ser visto na tabela gerada abaixo.

In [25]:
pagerank_result.sort_values('PAGE_RANK', ascending=False).head()

,NODE_ID,PAGE_RANK
0,1,0.000139
120,202,0.000125
94,144,0.000117
887,3996,0.000093
182,361,0.000092


### 3) Como você poderia usar o PageRank caso você fosse um investidor em bitcoins?